In [ ]:
# Import the subprocess, pandas, and numpy libraries
import subprocess
import pandas as pd
import numpy as np

# Create a list of the four DNA bases
bases=["C","T","G","A"]

#Perform tRNA search for the first (delta ribozymes) false positive dataset
# Loop through the bases in the "bases" list
for bas in bases: 
    # Define the input command for tRNAscan-SE with the current base and file paths
    inp2="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_all_viral_sequences_v2.fasta -o /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_v2.txt -f /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_secondary_struc.txt"
    
    # Run the input command using the subprocess.run() function: Obtain the tRNAs in all phage contigs associated with drzs
    subprocess.run(inp2, shell=True)


#Perform tRNA search for the fourth (final motif, theta ribozymes) false positive dataset
# Loop through the bases in the "bases" list
for bas in bases: 
    # Define the input command for tRNAscan-SE with the current base and file paths
    inp2="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_all_viral_sequences_v2.fasta -o /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_v2.txt -f /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_secondary_struc.txt"
    
    # Run the input command using the subprocess.run() function: Obtain the tRNAs in all phage contigs associated with drzs
    subprocess.run(inp2, shell=True)

In [31]:
# Define a list of the four DNA bases
bases=["C","T","G","A"]

# Define a list of the four sequencing runs
run=["first","second","third","fourth"]

# Create an empty dictionary to store the results
wholedic={}

# Create sub-dictionaries for tRNA and ribozyme results, as well as a dictionary for the unique sequences
wholedic["trna"]={}
wholedic["ribo"]={}
wholedic["dfs"]={}

# Loop through the sequencing runs in the "run" list
for nr in run:
    # Create sub-dictionaries for tRNA and ribozyme results, as well as a dictionary for the unique sequences, for the current sequencing run
    wholedic["trna"][nr]={}
    wholedic["ribo"][nr]={}
    wholedic["dfs"][nr]={}
    
    # Loop through the bases in the "bases" list
    for bas in bases: 
        # Open the ribozyme results file for the current base and sequencing run
        rnarobo_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/"+nr+"/"+bas+"/comb_rnarobo_result_precise.txt")
        rnarobo=rnarobo_.readlines()
        rnarobo_.close()
        
        # If the current sequencing run is the fourth one, open the tRNA results file for the current base and sequencing run
        if nr=="fourth":
            trna_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/fourth/"+bas+"/combined_trna_scan_v2.txt")
            trna=trna_.readlines()
            trna_.close()
        # If the current sequencing run is not the fourth one, open the tRNA results file for the first sequencing run and the current base
        else:
            trna_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_v2.txt")
            trna=trna_.readlines()
            trna_.close()

        # Create a dictionary with tRNA sequence names as keys and starting positions as values
        dicti={}
        for line in trna[3:]:
            line=line.strip().split()
            if line[0] in dicti:
                dicti[line[0]].append(int(line[3]))
            else:
                dicti[line[0]]=[int(line[3])]

        # Create a dictionary with ribozyme sequence names as keys and starting positions as values, and a list of unique ribozyme sequences
        dicti_r={}
        ribis=[]
        for line in rnarobo[11:-6]:
            if line[0]!="|":
                line=line.replace(".","_")
                line=line.strip().split()
                if line[2] in dicti_r:
                    dicti_r[line[2]].append(int(line[0]))
                else:
                    dicti_r[line[2]]=[int(line[0])]
            # If the current line is a unique ribozyme sequence, append it to the list of unique ribozyme sequences
            if line[0]=="|":
                lin_stripped="".join(line.strip().split("|"))
                if lin_stripped not in ribis:
                    ribis.append(lin_stripped)
        
        # Store the tRNA and ribozyme dictionaries for the current base and sequencing run in the "wholedic" dictionary
        wholedic["trna"][nr][bas]=dicti
        wholedic["ribo"][nr][bas]=dicti_r
        
        # Find the ribozymes that match with tRNAs within a certain range of positions
        matching=[]
        for key in dicti_r:
            if key in dicti:
                for value in dicti_r[key]:
                    for value2 in dicti[key]:
                        if value-value2 in range(-5,6):
                            matching.append([key,value,value2])
        
        # Remove duplicate matches
        newm=[]
        for mat in matching:
            if mat not in newm:
                newm.append(mat)
        
        # Create a DataFrame to store the results
        lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])
        check=0
        duplicates=[]
        for numbr in range(len(newm)):
            match=newm[numbr]
            lin_df=""
            for line in trna[3:]:
                line=line.strip().split()
                if match[0]==line[0]:
                    if len(lin_df.split())==0:
                        if str(match[2])==line[3]:
                            lin_df+=" ".join(line)+" "
                            if lin_df[-2]!="o":
                                lin_df+="NA "
                    else: 
                        duplicates.append(line)
            for line in rnarobo[11:-6]:
                line=line.replace(".","_")
                line=line.strip().split()
                if len(lin_df.split())<13:
                    if check==1:
                        lin_df+=str(line[0])
                        check=0
                    if str(match[1])==line[0]:
                        lin_df+=" ".join(line[:2])
                        lin_df+=" "
                        check=1
            lin_f=lin_df.split()
            lets_do_this.loc[numbr]=np.array(lin_f)
        
        # Store the unique ribozyme sequences for the current base and sequencing run in the "wholedic" dictionary
        wholedic["dfs"][nr][bas]=ribis
        
        # Print the number of unique delta ribozymes and unique trna associated ribozyme sequences found for the current base and sequencing run
        ln=0
        for x in dicti_r:
            ln+=len(dicti_r[x])
        print(nr, bas,len(ribis),len(lets_do_this["rnarobo_seq"].unique()))

first C 840 105
first T 424 49
first G 301 2
first A 284 12
second C 508 97
second T 160 38
second G 48 2
second A 64 2
third C 193 76
third T 42 30
third G 3 1
third A 5 1
fourth C 302 127
fourth T 53 34
fourth G 8 5
fourth A 11 3


In [ ]:
#Check which of the unique sequences are foun in both the true als well als the false positive search. If present in both, count them as true positives
# Define a list of the four sequencing runs
run=["first","second","third","fourth"]

# Define a list of the three DNA bases other than "C"
bases=["T","G","A"]

#check for the duplicate hits (e.g. false positives which where found in both CNRA* as well as TNRA*)
for nr in run:
    for bas in bases[:]:
        dictiT_r=wholedic["dfs"][nr][bas]
        dicti_r=wholedic["dfs"][nr]["C"]
        newm=[]
        #now lets see which of the unique ribozymes of the dataframe are also found in the C one
        for y in set(dictiT_r):
            if y in set(dicti_r):
                newm.append(y)
        print(nr,bas,len(newm))



#to get the final number of unique sequences identified from each motif, we need to subtract the number of "duplicates" from the number of unique ribozymes obtained prior